In [1]:
# Unicode Handling
from __future__ import unicode_literals
import codecs

import numpy as np
import gensim

# spacy is used for pre-processing and traditional NLP
import spacy
from spacy.en import English

# Gensim is used for LDA and word2vec
from gensim.models.word2vec import Word2Vec

In [2]:
# Unicode Handling
from __future__ import unicode_literals

import pandas as pd
import json

data = pd.read_csv("../../../lesson-13/assets/dataset/stumbleupon.tsv", sep='\t',
                  encoding="utf-8")
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,24,0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,40,0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,24,0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,14,0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...


In [4]:
body = data['body'].dropna()

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary = 'False',stop_words = 'english',min_df=3)

docs = cv.fit_transform(body)

id2word = dict(enumerate(cv.get_feature_names()))

In [5]:
print id2word

{0: u'00', 1: u'000', 2: u'000000', 3: u'001', 4: u'007', 5: u'00am', 6: u'00pm', 7: u'01', 8: u'01pm', 9: u'02', 10: u'0206790666', 11: u'025', 12: u'03', 13: u'04', 14: u'044', 15: u'05', 16: u'06', 17: u'0674921071', 18: u'07', 19: u'075', 20: u'0782835788', 21: u'08', 22: u'09', 23: u'0g', 24: u'0http', 25: u'0px', 26: u'0s', 27: u'0sodium', 28: u'10', 29: u'100', 30: u'1000', 31: u'100000000000000000', 32: u'1000000000000000000', 33: u'1000px', 34: u'1000s', 35: u'1001', 36: u'10013', 37: u'100g', 38: u'100k', 39: u'100m', 40: u'100ml', 41: u'100px', 42: u'100th', 43: u'101', 44: u'10184', 45: u'102', 46: u'1024', 47: u'103', 48: u'1034', 49: u'1036', 50: u'104', 51: u'105', 52: u'10522', 53: u'10529', 54: u'106', 55: u'107', 56: u'108', 57: u'1080p', 58: u'109', 59: u'1090', 60: u'10am', 61: u'10g', 62: u'10km', 63: u'10lbs', 64: u'10m', 65: u'10mm', 66: u'10oz', 67: u'10pm', 68: u'10px', 69: u'10th', 70: u'11', 71: u'110', 72: u'1100', 73: u'11000', 74: u'1107748553', 75: u'111'

In [6]:
from gensim.models.ldamodel import LdaModel
from gensim.matutils import Sparse2Corpus

#What does this do?
#Convert our sparse matrix into a gensim corpus stream
corpus = Sparse2Corpus(docs, documents_columns = False)

#We need to pass in the dictionary so that way, we can map our integers that the LDA model works with back to
#the original vocab word.
lda_model = LdaModel(corpus = corpus, id2word = id2word,num_topics = 10)

In [9]:
type(corpus)

gensim.matutils.Sparse2Corpus

In [7]:
num_topics = 100
n_words_per_topic = 5
for ti, topic in enumerate(lda_model.show_topics(num_topics = num_topics,num_words = n_words_per_topic,formatted = True)):
    print 'Topic: %d' % ti
    print topic

Topic: 0
(0, u'0.003*make + 0.003*just + 0.003*like + 0.003*new + 0.003*time')
Topic: 1
(1, u'0.003*like + 0.003*just + 0.003*make + 0.003*time + 0.002*style')
Topic: 2
(2, u'0.004*sports + 0.003*like + 0.003*photo + 0.003*time + 0.002*world')
Topic: 3
(3, u'0.003*new + 0.003*like + 0.002*food + 0.002*make + 0.002*ll')
Topic: 4
(4, u'0.003*time + 0.003*just + 0.003*like + 0.002*make + 0.002*think')
Topic: 5
(5, u'0.004*minutes + 0.004*add + 0.004*just + 0.004*make + 0.004*recipe')
Topic: 6
(6, u'0.003*like + 0.003*new + 0.003*just + 0.002*people + 0.002*com')
Topic: 7
(7, u'0.002*people + 0.002*health + 0.002*time + 0.002*help + 0.002*year')
Topic: 8
(8, u'0.006*sugar + 0.006*butter + 0.006*bake + 0.005*recipe + 0.005*vanilla')
Topic: 9
(9, u'0.004*recipe + 0.004*recipes + 0.004*make + 0.003*just + 0.003*food')


#### Using Word2Vec

In [8]:
from gensim.models.word2vec import Word2Vec

In [9]:
text = data.body.dropna().map(lambda x: x.split())

In [10]:
print text

0       [A, sign, stands, outside, the, International,...
1       [And, that, can, be, carried, on, a, plane, wi...
2       [Apples, The, most, popular, source, of, antio...
3       [There, was, a, period, in, my, life, when, I,...
4       [Jersey, sales, is, a, curious, business, Whet...
5       [Genital, herpes, is, caused, by, herpes, simp...
6       [Our, favorite, summer, holiday, is, just, aro...
7       [Racing, For, Recovery, is, the, growing, idea...
8       [Resolutions, are, for, suckers, Instead, of, ...
9       [More, brownies, It, seems, that, I, can, t, g...
10      [Faced, with, a, stream, of, financial, scanda...
11      [As, you, surely, know, by, now, Brett, Favre,...
12      [There, are, the, 700, different, types, of, g...
13                                                     []
14      [Really, this, is, just, an, excuse, to, get, ...
15      [TM, 2012, Turner, Broadcasting, System, Inc, ...
16      [Link, We, re, pretty, skeptical, about, walki...
17      [The, 

In [12]:
#Min_count argument only keeps those words that occur at least 5 times in a corpus. Other words are probably garbage.
model = Word2Vec(text,size = 100,window = 5,min_count = 5, workers = 4)
model.most_similar(positive = ['cookie'],negative = ['cookie'])

[(u'Snapshot', 0.0),
 (u'Mitts', 0.0),
 (u'antagonists', 0.0),
 (u'han', 0.0),
 (u'Calls', 0.0),
 (u'nicest', 0.0),
 (u'openly', 0.0),
 (u'Barrel', 0.0),
 (u'numericId', 0.0),
 (u'consoles', 0.0)]

In [2]:
# Loading the tweet data
filename = '../../assets/dataset/captured-tweets.txt'
tweets = []
for tweet in codecs.open(filename, 'r', encoding="utf-8"):
    tweets.append(tweet)
# Setting up spacy
nlp_toolkit = English()

## Exercise 1a

Write a function that can take a take a sentence parsed by `spacy` and identify if it mentions a company named 'Google'. Remember, `spacy` can find entities and codes them as `ORG` if they are a company. Look at the slides for class 13 if you need a hint:

### Bonus (1b)

Parameterize the company name so that the function works for any company.

In [51]:
parsed_sentence = nlp_toolkit('Google drives a car that runs on a wheel')
type(parsed_sentence)

spacy.tokens.doc.Doc

In [39]:
#Parsing in Spacy is of type Document

def contains_google(parsed_sentence):
    
    for word in parsed_sentence:
        print word
        if (word.ent_type_ == 'ORG') and (str(word) == 'Google'):
            return True
    
    return False

In [40]:
contains_google(parsed_sentence)

hello
Google


True

In [35]:
for word in parsed_sentence:
    print str(word) == 'Google'

False
True


In [58]:
#def mentions_company(parsed):
    # Return True if the sentence contains an organization and that organization is Google
#   for entity in parsed.ents:
#        print entity
#        if entity == 'Google':
#            return True
#    return False
    #return False

# 1b

def mentions_company(parsed, company='Google'):
    for word in parsed:
        if (word.ent_type_ == 'ORG') and word.text == company:
            return True
    return False    
    

In [59]:
mentions_company(parsed_sentence)

True

## Exercise 1c

Write a function that can take a sentence parsed by `spacy` 
and return the verbs of the sentence (preferably lemmatized)

In [52]:
def get_actions(parsed):
    actions = []
    
    for word in parsed:
        if word.pos_ == 'VERB':
            actions.append(word.text)
    return actions

In [53]:
get_actions(parsed_sentence)

[u'drives', u'runs']

In [60]:
'drives' in get_actions(parsed_sentence)

True

## Exercise 1d
For each tweet, parse it using spacy and print it out if the tweet has 'release' or 'announce' as a verb. You'll need to use your `mentions_company` and `get_actions` functions.

In [ ]:
for tweet in tweets:
    parsed = nlp_toolkit(tweet)
    
    verbs = get_actions(parsed)
    
    if ('release' in verbs or 'announce' in verbs) and mentions_company(parsed):
        print tweet
        


## Exercise 1e
Write a function that identifies countries - HINT: the entity label for countries is GPE (or GeoPolitical Entity)



In [ ]:
def mentions_country(parsed, country):
    pass


## Exercise 1f

Re-run (d) to find country tweets that discuss 'Iran' announcing or releasing.


In [ ]:
for tweet in tweets:
    parsed = nlp_toolkit(tweet)
    pass

## Exercise 2
Build a `word2vec` model of the tweets we have collected using `gensim`.

### Exercise 2a:
First take the collection of tweets and tokenize them using spacy.

* Think about how this should be done. 
* Should you only use upper-case or lower-case? 
* Should you remove punctuations or symbols? 

In [ ]:
text_split = [[x.text if x.pos != spacy.parts_of_speech.VERB else x.lemma_ 
                for x in nlp_toolkit(t)] for t in tweets]


### Exercise 2b:
Build a `word2vec` model.
Test the window size as well - this is how many surrounding words need to be used to model a word. What do you think is appropriate for Twitter? 

In [ ]:
model = Word2Vec(text_split, size=100, window=4, min_count=5, workers=4)

### Exercise 2c:
Test your word2vec model with a few similarity functions. 
* Find words similar to 'Syria'.
* Find words similar to 'war'.
* Find words similar to "Iran".
* Find words similar to 'Verizon'. 



In [ ]:
model.most_similar(positive=['Syria'])

# Exercise 2d

Adjust the choices / parameters in (b) and (c) as necessary.


## Exercise 3

Filter tweets to those that mention 'Iran' or similar entities and 'war' or similar entities.
* Do this using just spacy.
* Do this using word2vec similarity scores.

In [ ]:
# Using spacy
for tweet in tweets:
    parsed = nlp_toolkit(tweet)
    pass

In [ ]:
# Using word2vec similarity scores
for tweet in tweets[:200]:
    parsed = nlp_toolkit(tweet)
    pass
